In [1]:
%%HTML
<!-- Load d3 library -->
<script src="d3.min.js"></script>

# D3 Introduction

*[D3.js](http://d3js.org/) is a JavaScript library for **manipulating documents** based on **data**.*

D3 started its life as a data visualization library developed by [Mike Bostock](https://twitter.com/mbostock). At its core, D3 facilitates interactions with the DOM and allows you to effectively map data to DOM elements.

Some important characteristics of D3:

* It simplifies your access to the DOM but never obscures the DOM behind an abstraction.
* You need strong HTML, CSS, optionally SVG and Canvas knowledge to be effective with D3.
* The data model is **crucial**. A model that allows a simpler mapping to the DOM hugely simplifies the task.

## Let's make a table

In [2]:
%%HTML

<style>
.col { display: inline-block; width: 200px; height: 30px; padding: 3px;}
.row.even { background: lightgray; }
#Table1 { margin: 10px 0 10px 0; }
</style>

<div id="Table1" />

<script>
function Person(name, sex, height, weight) {
    this.name = name;
    this.sex = sex;
    this.height = height;
    this.weight = weight;
}

function fantasyChars() {
    return [
        new Person('Peter Parker', 'M', 178, 76),
        new Person('Anakin Skywalker', 'M', 185, 84),
        new Person('Galadriel', 'F', 193, undefined),
        new Person('Tyrion Lannister', 'M', 95, undefined)
    ];
    
}

var renderTable = function (people) {
    d3.select('#Table1').selectAll('.row').data(people)
        .enter()
        .append('div').attr('class', 'row')
        .classed('even', function (d, i) { return i % 2 === 0;})
        .html(function (d) {
            return '<span class="col">' + d.name + '</span>' +
                '<span class="col">' + d.sex + '</span>' +
                '<span class="col">' + d.height + '</span>' +
                '<span class="col">' + d.weight + '</span>';
        });    
}

renderTable(fantasyChars());
</script>

#### What happenned?

* We defined a data collection and bound it to all the `.row` elements inside #Table1.
* `enter()` defined what to do when **new** data is encountered.
* D3 uses a fluent interface similar to jQuery.

Let's play a little bit with the code:

* Add `.row { background: darkgray; }` class?
* Add/remove some data?

In [3]:
%%javascript

// Let's add a new person
var misty = fantasyChars();
misty.push(new Person('Vin', 'F', undefined, undefined));
renderTable(misty);

<IPython.core.display.Javascript object>

In [4]:
%%javascript

// What happens if we remove a person?
var removed = fantasyChars();
removed.pop();
renderTable(removed);

<IPython.core.display.Javascript object>

#### Why adding a row works, but not removing?

The contract for data removal was not defined.

D3 defines the `data-join` contract for 3 cases:

* `enter()` for new data elements
* `exit()` for removed data elements
* Direct operations on the data binding for updated data elements

The convention for enter, update and exit:

```javascript
//bind data and save the reference
var dataBinding = d3.select(parent).selectAll(boundElement).data(data_to_bind);

// update only
dataBinding.style(...).attr(...);

// enter only
dataBinding.enter().append(...).attr(...);

// common section for enter + update
dataBinding.style(...).attr(...);

// exit only
dataBinding.exit().remove();
```

## Let's make a table with enter() and exit()

In [5]:
%%javascript

renderTable = function (people) {
    var tableBinding = d3.select('#Table1').selectAll('.row').data(people);
    
    tableBinding.enter()
        .append('div').attr('class', 'row')
        .classed('even', function (d, i) { return i % 2 === 0;})
        .html(function (d) {
            return '<span class="col">' + d.name + '</span>' +
                '<span class="col">' + d.sex + '</span>' +
                '<span class="col">' + d.height + '</span>' +
                '<span class="col">' + d.weight + '</span>';
        });
    
    tableBinding.exit().remove();
}

var removed = fantasyChars();
removed.pop();
renderTable(removed);

<IPython.core.display.Javascript object>

## Let's make a table with enter(), exit() and update

In [6]:
%%HTML

<script>
renderTable2 = function (people) {
    var tableBinding = d3.select('#Table1').selectAll('.row').data(people);
        
    tableBinding.enter()
        .append('div').attr('class', 'row')
        .classed('even', function (d, i) { return i % 2 === 0;});
    
    tableBinding.html(function (d) {
            return '<span class="col">' + d.name + '</span>' +
                '<span class="col">' + d.sex + '</span>' +
                '<span class="col">' + d.height + '</span>' +
                '<span class="col">' + d.weight + '</span>';
        });

    tableBinding.exit().remove();

};
</script>

In [7]:
%%javascript

var modified = fantasyChars();
modified[1].name = 'Darth Vader';
// this doesn't do anything since we haven't defined the operations for update
renderTable2(modified);

<IPython.core.display.Javascript object>

The function `selection.html` makes it difficult to compose additional functions in D3. It should be used with care.

# Let's make a bar chart

In [8]:
%%HTML

<style>
.bar { position: absolute; width: 50px; background: blue;}
.label { position: absolute; color: lightgray; font-size: 15px; transform: rotate(-90deg);
         transform-origin: top left; bottom: -25px; left: 15px; }
#Table2 { margin: 10px 0 10px 0; height: 400px; position: relative;}
</style>

<div id="Table2" />

<script>
var renderBar = function (people) {
    function barHeight(d) {
        return 400/250 * d.height;
    }
    
    var barBinding = d3.select('#Table2').selectAll('.bar').data(people);
    var barEnter = barBinding.enter();
    
    barEnter.append('div').attr('class', 'bar')
        .style('top', function (d, i) {
            return (400 - barHeight(d)) + 'px';
        })
        .style('left', function (d, i) {
            return (30 + (50 + 30) * i) + 'px';
        })
        .style('height', function (d, i) {
            return barHeight(d) + 'px';
        })
        .append('div').attr('class', 'label')
        .text(function (d) { return d.name; });
    
    barBinding.exit().remove();
}

renderBar(fantasyChars());
</script>

We can add the update contract and transition to the bar char.

In [9]:
%%javascript

renderBar = function (people) {
    function barHeight(d) {
        return 400/250 * d.height;
    }
    
    var barBinding = d3.select('#Table2').selectAll('.bar').data(people, function (d) { return d.name; });
            
    var barEnter = barBinding.enter().append('div').attr('class', 'bar');
    
    barEnter.append('div').attr('class', 'label');

    barBinding.transition().duration(2000)
        .style('top', function (d, i) {
            return (400 - barHeight(d)) + 'px';
        })
        .style('left', function (d, i) {
            console.log(d);
            return (30 + (50 + 30) * i) + 'px';
        })
        .style('height', function (d, i) {
            return barHeight(d) + 'px';
        });

    barBinding.select('.label').text(function (d) { return d.name; });
    
    barBinding.exit().remove();
}

<IPython.core.display.Javascript object>

In [10]:
%%javascript

var wheel = fantasyChars();
wheel.push(new Person('al\'Lan Mandragoran', 'M', 200, undefined));

renderBar(wheel);

<IPython.core.display.Javascript object>

In [11]:
%%javascript

var males = fantasyChars().filter(function (c) { return c.name !== 'Galadriel'; });

renderBar(males);

<IPython.core.display.Javascript object>

This is only the beginning. Explore the [vast](https://github.com/mbostock/d3/wiki/Gallery) [examples](http://bl.ocks.org/mbostock) [online](https://www.jasondavies.com/) to get a taste of what D3 can do.

Happy hacking, and do keep me posted with your work!